<a href="https://colab.research.google.com/github/nguyenanhtienabcd/M02_PROJECT_TEXT_CLASSIFICATION/blob/feature%2Ftext_classifications/M02_Project_Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder as le

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!gdown 1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R

Downloading...
From: https://drive.google.com/uc?id=1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R
To: /content/2cls_spam_text_cls.csv
100% 486k/486k [00:00<00:00, 66.7MB/s]


In [3]:
FILE_PATH = '/content/2cls_spam_text_cls.csv'
df = pd.read_csv(FILE_PATH)
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
messages = df['Message'].tolist()
labels = df['Category'].tolist()

# 1. Tạo Hàm tiền xử lý dữ liệu ban đầu

## Tạo một hàm lowercase

In [5]:
def lowercase(text):
  return text.lower()

## Loại bỏ các dấu và chấm trong câu

In [6]:
def punctuation_removal(text):
  translator = str.maketrans('', '', string.punctuation)
  return text.translate(translator)

## Tách các từ trong văn bản ra

In [7]:
def tokenize(text):
  return nltk.word_tokenize(text)

## Loại bỏ stop words

In [8]:
def remove_stopwords(tokens):
  # lấy một set các từ stopword trong tiếng anh như: a, an , the, and...
  stop_words = set(nltk.corpus.stopwords.words('english'))
  # trả về một danh sách các từ không chứa stopword
  return [word for word in tokens if word not in stop_words]

## Đưa các động từ và danh từ số nhiều về từ gốc (về từ hiện tại đơn)

In [9]:
def stemming(tokens):
  stemmer = nltk.stem.PorterStemmer()
  return [stemmer.stem(word) for word in tokens]

## Xây dựng hàm tiền xử lý dữ liệu từ các hàm trên

In [10]:
def preprocess_text(text):
  text = lowercase(text)
  text = punctuation_removal(text)
  tokens = nltk.word_tokenize(text)
  tokens = remove_stopwords(tokens)
  tokens = stemming(tokens)
  return tokens

# 2. Tạo từ điển

In [11]:
message = [preprocess_text(text) for text in messages]
label = labels

In [12]:
def create_dictionary(messages):
  dictionary = []
  for tokens in messages:
    for token in tokens:
      if token not in dictionary:
        dictionary.append(token)
  return dictionary

In [13]:
dictionary = create_dictionary(message)

# 3. Tạo đặc tính: xác định số lần lặp của các từ trong một đoạn lấy "vector từ" của của từ điển làm chuẩn

In [14]:
def create_features(tokens, dictionary):
  features = np.zeros(len(dictionary))
  for token in tokens:
    if token in dictionary:
      features[dictionary.index(token)] += 1
  return features

In [15]:
X = np.array([create_features(tokens, dictionary) for tokens in message])
y = np.array(label)

# 4. Tách dữ liệu thành các phần: train, val, test

In [16]:
VAL_SIZE = 0.2
TEST_SIZE = 0.125
SEED = 0
SHUFFLE = True

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=VAL_SIZE,
    random_state=SEED,
    shuffle=SHUFFLE
    )
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train,
    test_size=TEST_SIZE,
    random_state=SEED,
    shuffle=SHUFFLE
    )

In [17]:
print(X_train.shape[0])
print(X_val.shape[0])
print(X_test.shape[0])

3899
1115
558


# 5. Tạo model training

In [18]:
%%time
model = GaussianNB()
print('start training...')
model.fit(X_train, y_train)
print('training done!')

start training...
training done!
CPU times: user 356 ms, sys: 174 ms, total: 531 ms
Wall time: 526 ms


# 6. kiểm tra model

In [19]:
y_val_predicted = model.predict(X_val)
y_test_predicted = model.predict(X_test)
val_accuracy = accuracy_score(y_val, y_val_predicted)
test_accuracy = accuracy_score(y_test, y_test_predicted)
print(f'Validation Accuracy: {val_accuracy}')
print(f'Test Accuracy: {test_accuracy}')

Validation Accuracy: 0.8816143497757848
Test Accuracy: 0.8602150537634409


# 7. Tạo một hàm dự đoán

In [20]:
def predict_spam(text):
  tokens = preprocess_text(text)
  features = create_features(tokens, dictionary)
  prediction = model.predict([features])
  # chuyển đổi nhãn mã hóa thành nhãn gốc
  return prediction[0]
#

In [21]:
text = 'tommorow, can we go to the park'
predict_spam(text)

'ham'